In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import geopandas
import numpy as np
import glob
from datetime import datetime
import time

#from geodata_functions import *
import re

# Process time series confirmed

In [139]:
df = pd.read_csv(r'C:\Users\Wan Xin\Documents\NUS\Information Visualization\Assignments\COVID data\time_series_covid_19_confirmed.csv')

In [140]:
# correct UK to United Kingdom
uk_idx = df['Country'] == 'UK'
df.loc[uk_idx, 'Country'] = 'United Kingom'

#correct Mainland China to China
cn_idx = df['Country'] == 'Mainland China'
df.loc[cn_idx, 'Country'] = 'China'

#correct US to United States
us_idx = df['Country'] == 'US'
df.loc[us_idx, 'Country'] = 'United States of America'


In [142]:
df.head()

,Province/State,Country,Lat,Long,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,...,2/11/2020,2/12/2020,2/13/2020,2/14/2020,2/15/2020,2/16/2020,2/17/2020,2/18/2020,2/19/2020,2/20/2020
0,Anhui,China,31.82571,117.2264,1,9,15,39,60,70,...,860,889,910,934,950,962,973,982,986,987
1,Beijing,China,40.18238,116.4142,14,22,36,41,68,80,...,342,352,366,372,375,380,381,387,393,395
2,Chongqing,China,30.05718,107.8740,6,9,27,57,75,110,...,505,518,529,537,544,551,553,555,560,567
3,Fujian,China,26.07783,117.9895,1,5,10,18,35,59,...,267,272,279,281,285,287,290,292,293,293
4,Gansu,China,36.06110,103.8343,0,2,2,4,7,14,...,86,87,90,90,90,90,91,91,91,91


In [21]:
#dates = df.iloc[:,4:-1]
dates = []
for col in df.iloc[:,4:-1].columns: 
    dates.append(col)
    
dates

['1/22/2020',
 '1/23/2020',
 '1/24/2020',
 '1/25/2020',
 '1/26/2020',
 '1/27/2020',
 '1/28/2020',
 '1/29/2020',
 '1/30/2020',
 '1/31/2020',
 '2/1/2020',
 '2/2/2020',
 '2/3/2020',
 '2/4/2020',
 '2/5/2020',
 '2/6/2020',
 '2/7/2020',
 '2/8/2020',
 '2/9/2020',
 '2/10/2020',
 '2/11/2020',
 '2/12/2020',
 '2/13/2020',
 '2/14/2020',
 '2/15/2020',
 '2/16/2020',
 '2/17/2020',
 '2/18/2020',
 '2/19/2020',
 '2/20/2020']

In [49]:
#aggregating countries
ncov_time = df.groupby(['Country'])[dates].sum()
ncov_time = ncov_time.reset_index()
ncov_time

,Country,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,...,2/11/2020,2/12/2020,2/13/2020,2/14/2020,2/15/2020,2/16/2020,2/17/2020,2/18/2020,2/19/2020,2/20/2020
0,Australia,0,0,0,0,4,5,5,6,9,...,15,15,15,15,15,15,15,15,15,15
1,Belgium,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
2,Cambodia,0,0,0,0,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,Canada,0,0,0,0,1,1,2,2,2,...,7,7,7,7,7,7,8,8,8,8
4,China,547,639,916,1399,2062,2863,5494,6070,8124,...,44327,44699,59832,66292,68347,70446,72364,74139,74546,74999
5,Egypt,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,1,1,1,1,1
6,Finland,0,0,0,0,0,0,0,1,1,...,1,1,1,1,1,1,1,1,1,1
7,France,0,0,2,3,3,3,4,5,5,...,11,11,11,11,12,12,12,12,12,12
8,Germany,0,0,0,0,0,1,4,4,4,...,16,16,16,16,16,16,16,16,16,16
9,Hong Kong,0,2,2,5,8,8,8,10,10,...,49,50,53,56,56,57,60,62,63,68


In [126]:
#Merge to get all countries fo
dataframe = dataframe.rename(columns={'properties.name': 'Country'})
world_ncov = pd.merge(ncov_time, dataframe[['Country','id']], on=['Country'], how='outer')
world_ncov

,Country,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,...,2/12/2020,2/13/2020,2/14/2020,2/15/2020,2/16/2020,2/17/2020,2/18/2020,2/19/2020,2/20/2020,id
0,Australia,0.0,0.0,0.0,0.0,4.0,5.0,5.0,6.0,9.0,...,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,AUS
1,Belgium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,BEL
2,Cambodia,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,KHM
3,Canada,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,2.0,...,7.0,7.0,7.0,7.0,7.0,8.0,8.0,8.0,8.0,CAN
4,China,547.0,639.0,916.0,1399.0,2062.0,2863.0,5494.0,6070.0,8124.0,...,44699.0,59832.0,66292.0,68347.0,70446.0,72364.0,74139.0,74546.0,74999.0,CHN
5,Egypt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,EGY
6,Finland,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,FIN
7,France,0.0,0.0,2.0,3.0,3.0,3.0,4.0,5.0,5.0,...,11.0,11.0,11.0,12.0,12.0,12.0,12.0,12.0,12.0,FRA
8,Germany,0.0,0.0,0.0,0.0,0.0,1.0,4.0,4.0,4.0,...,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,DEU
9,Hong Kong,0.0,2.0,2.0,5.0,8.0,8.0,8.0,10.0,10.0,...,50.0,53.0,56.0,56.0,57.0,60.0,62.0,63.0,68.0,NaN


In [137]:
world_lines = geopandas.read_file(r'C:\Users\Wan Xin\Documents\Data\Earth shapefiles\ne_50m_admin_0_countries\ne_50m_admin_0_countries.shp')
world = world_lines[(world_lines['POP_EST'] > 0) & (world_lines['ADMIN'] != 'Antarctica')]
world = world.rename(columns={'ADMIN': 'Country'})

#merge to get all countries and fill with zeros to make map in d3
world_ncov = pd.merge(ncov_time, world[['Country','ADM0_A3']], on=['Country'], how='outer')
world_ncov= world_ncov.fillna(0)
world_ncov

,Country,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,...,2/12/2020,2/13/2020,2/14/2020,2/15/2020,2/16/2020,2/17/2020,2/18/2020,2/19/2020,2/20/2020,ADM0_A3
0,Australia,0.0,0.0,0.0,0.0,4.0,5.0,5.0,6.0,9.0,...,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,AUS
1,Belgium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,BEL
2,Cambodia,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,KHM
3,Canada,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,2.0,...,7.0,7.0,7.0,7.0,7.0,8.0,8.0,8.0,8.0,CAN
4,China,547.0,639.0,916.0,1399.0,2062.0,2863.0,5494.0,6070.0,8124.0,...,44699.0,59832.0,66292.0,68347.0,70446.0,72364.0,74139.0,74546.0,74999.0,CHN
5,Egypt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,EGY
6,Finland,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,FIN
7,France,0.0,0.0,2.0,3.0,3.0,3.0,4.0,5.0,5.0,...,11.0,11.0,11.0,12.0,12.0,12.0,12.0,12.0,12.0,FRA
8,Germany,0.0,0.0,0.0,0.0,0.0,1.0,4.0,4.0,4.0,...,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,DEU
9,Hong Kong,0.0,2.0,2.0,5.0,8.0,8.0,8.0,10.0,10.0,...,50.0,53.0,56.0,56.0,57.0,60.0,62.0,63.0,68.0,0


In [138]:
#saving the csv
world_ncov.to_csv(r'input path here')

# Process 2019 COVID data

In [2]:
df = pd.read_csv(r'C:\Users\Wan Xin\Documents\NUS\Information Visualization\Assignments\COVID data\2019_nCoV_data.csv')

In [3]:
df.head()

,Sno,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020 12:00:00,Anhui,China,01/22/2020 12:00:00,1,0,0
1,2,01/22/2020 12:00:00,Beijing,China,01/22/2020 12:00:00,14,0,0
2,3,01/22/2020 12:00:00,Chongqing,China,01/22/2020 12:00:00,6,0,0
3,4,01/22/2020 12:00:00,Fujian,China,01/22/2020 12:00:00,1,0,0
4,5,01/22/2020 12:00:00,Gansu,China,01/22/2020 12:00:00,0,0,0


In [8]:
df['Date'] = pd.to_datetime(df['Date'])
df['Date'] = df['Date'].dt.date

In [9]:
time_total = df.groupby(['Date'])[['Confirmed','Deaths','Recovered']].sum()

In [10]:
time_total

,Confirmed,Deaths,Recovered
Date,,,
2020-01-22,555,0,0
2020-01-23,653,18,30
2020-01-24,941,26,36
2020-01-25,2019,56,49
2020-01-26,2794,80,54
2020-01-27,4473,107,63
2020-01-28,6057,132,110
2020-01-29,7783,170,133
2020-01-30,9776,213,187


In [11]:
time_total.to_csv(r'C:\Users\Wan Xin\Documents\NUS\Information Visualization\Assignments\COVID data\time_total.csv')

# Process line list data

In [26]:
df = pd.read_csv(r'C:\Users\Wan Xin\Documents\NUS\Information Visualization\Assignments\COVID data\COVID19_line_list_data.csv')

In [28]:
df

,id,case_in_country,reporting date,Unnamed: 3,summary,location,country,gender,age,symptom_onset,...,hosp_visit_date,exposure_start,exposure_end,visiting Wuhan,from Wuhan,death,recovered,symptom,source,link
0,1,NaN,1/20/2020,NaN,First confirmed imported COVID-19 pneumonia pa...,"Shenzhen, Guangdong",China,male,66.0,1/3/2020,...,1/11/2020,12/29/2019,1/4/2020,1,0.0,0,0,NaN,Shenzhen Municipal Health Commission,http://wjw.sz.gov.cn/wzx/202001/t20200120_1898...
1,2,NaN,1/20/2020,NaN,First confirmed imported COVID-19 pneumonia pa...,Shanghai,China,female,56.0,1/15/2020,...,1/15/2020,NaN,1/12/2020,0,1.0,0,0,NaN,Official Weibo of Shanghai Municipal Health Co...,https://www.weibo.com/2372649470/IqogQhgfa?fro...
2,3,NaN,1/21/2020,NaN,First confirmed imported cases in Zhejiang: pa...,Zhejiang,China,male,46.0,1/4/2020,...,1/17/2020,NaN,1/3/2020,0,1.0,0,0,NaN,Health Commission of Zhejiang Province,http://www.zjwjw.gov.cn/art/2020/1/21/art_1202...
3,4,NaN,1/21/2020,NaN,new confirmed imported COVID-19 pneumonia in T...,Tianjin,China,female,60.0,NaN,...,1/19/2020,NaN,NaN,1,0.0,0,0,NaN,人民日报官方微博,https://m.weibo.cn/status/4463235401268457?
4,5,NaN,1/21/2020,NaN,new confirmed imported COVID-19 pneumonia in T...,Tianjin,China,male,58.0,NaN,...,1/14/2020,NaN,NaN,0,0.0,0,0,NaN,人民日报官方微博,https://m.weibo.cn/status/4463235401268457?
5,6,NaN,1/21/2020,NaN,First confirmed imported COVID-19 pneumonia pa...,Chongqing,China,female,44.0,1/15/2020,...,NaN,NaN,NaN,0,1.0,0,0,NaN,Chongqing Municipal Health Commission,http://wsjkw.cq.gov.cn/tzgg/20200121/249730.html
6,7,NaN,1/21/2020,NaN,First confirmed imported COVID-19 pneumonia pa...,Sichuan,China,male,34.0,1/11/2020,...,NaN,NaN,NaN,0,1.0,0,0,NaN,央视新闻,https://m.weibo.cn/status/4463300522087848?
7,8,NaN,1/21/2020,NaN,new confirmed imported COVID-19 pneumonia pati...,Beijing,China,male,37.0,1/14/2020,...,1/20/2020,1/10/2020,1/11/2020,1,0.0,0,0,NaN,Beijing Municipal Health Commission,http://wjw.beijing.gov.cn/xwzx_20031/wnxw/2020...
8,9,NaN,1/21/2020,NaN,new confirmed imported COVID-19 pneumonia pati...,Beijing,China,male,39.0,1/9/2020,...,1/14/2020,1/3/2020,1/4/2020,1,0.0,0,0,NaN,Beijing Municipal Health Commission,http://wjw.beijing.gov.cn/xwzx_20031/wnxw/2020...
9,10,NaN,1/21/2020,NaN,new confirmed imported COVID-19 pneumonia pati...,Beijing,China,male,56.0,1/16/2020,...,1/20/2020,1/8/2020,1/16/2020,1,0.0,0,0,NaN,Beijing Municipal Health Commission,http://wjw.beijing.gov.cn/xwzx_20031/wnxw/2020...


<h3> Let's analyze deaths </h3>

In [37]:
dead = df[df['death'] != '0']

In [56]:
dead.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63 entries, 48 to 1036
Data columns (total 21 columns):
id                       63 non-null int64
case_in_country          24 non-null float64
reporting date           63 non-null object
Unnamed: 3               0 non-null float64
summary                  63 non-null object
location                 63 non-null object
country                  63 non-null object
gender                   58 non-null object
age                      58 non-null float64
symptom_onset            43 non-null object
If_onset_approximated    42 non-null float64
hosp_visit_date          44 non-null object
exposure_start           1 non-null object
exposure_end             3 non-null object
visiting Wuhan           63 non-null int64
from Wuhan               63 non-null float64
death                    63 non-null object
recovered                63 non-null object
symptom                  9 non-null object
source                   63 non-null object
link           

<h4> Let's check if theres a dircrepancy in deaths in genders </h4>

In [72]:
#dead.groupby(['gender']).count()
dg = pd.DataFrame(dead.groupby('gender')['id'].nunique()).rename(columns={'id':'count'})
dg

,count
gender,
female,14
male,44


In [73]:
tg = pd.DataFrame(df.groupby('gender')['id'].nunique()).rename(columns={'id':'count'})
tg

,count
gender,
female,382
male,520


In [75]:
g = ((dg/tg)*100).set_index(tg.index).rename(columns={'count':'% of deaths'})
g

,% of deaths
gender,
female,3.664921
male,8.461538


<h4> Now let's take a look at the age to see which age range ar more susceptible to death </h4>

In [40]:
dead['age'].mean()

68.58620689655173

In [49]:
# find the number of deaths in each age range
ranges = [0,10,20,30,40,50,60,70,80,90,100]
death_age = dead.groupby(pd.cut(dead.age, ranges))['id'].count()
dcount_age = pd.DataFrame(death_age).rename(columns={'id':'count'})
dcount_age

,count
age,
"(0, 10]",0
"(10, 20]",0
"(20, 30]",0
"(30, 40]",3
"(40, 50]",3
"(50, 60]",9
"(60, 70]",18
"(70, 80]",11
"(80, 90]",14


In [50]:
#find the number of people in each  age group
age_count = df.groupby(pd.cut(df.age, ranges))['id'].count()
age_count = pd.DataFrame(age_count).rename(columns={'id':'count'})
age_count

,count
age,
"(0, 10]",19
"(10, 20]",15
"(20, 30]",105
"(30, 40]",149
"(40, 50]",131
"(50, 60]",183
"(60, 70]",134
"(70, 80]",72
"(80, 90]",33


In [54]:
j = ((dcount_age / age_count)*100).set_index(age_count.index).rename(columns={'count':'% of deaths'})
np.round(j, decimals=2)

,% of deaths
age,
"(0, 10]",0.00
"(10, 20]",0.00
"(20, 30]",0.00
"(30, 40]",2.01
"(40, 50]",2.29
"(50, 60]",4.92
"(60, 70]",13.43
"(70, 80]",15.28
"(80, 90]",42.42


<h3> Let's analyze Recovery </h3>

In [55]:
r = df[df['recovered'] != '0']

In [57]:
r.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159 entries, 194 to 1032
Data columns (total 21 columns):
id                       159 non-null int64
case_in_country          156 non-null float64
reporting date           159 non-null object
Unnamed: 3               0 non-null float64
summary                  159 non-null object
location                 159 non-null object
country                  159 non-null object
gender                   153 non-null object
age                      145 non-null float64
symptom_onset            103 non-null object
If_onset_approximated    103 non-null float64
hosp_visit_date          87 non-null object
exposure_start           23 non-null object
exposure_end             81 non-null object
visiting Wuhan           159 non-null int64
from Wuhan               159 non-null float64
death                    159 non-null object
recovered                159 non-null object
symptom                  42 non-null object
source                   159 non-null ob

In [58]:
r_age = r.groupby(pd.cut(r.age, ranges))['id'].count()
rcount_age = pd.DataFrame(r_age).rename(columns={'id':'count'})
rcount_age

,count
age,
"(0, 10]",10
"(10, 20]",3
"(20, 30]",21
"(30, 40]",33
"(40, 50]",28
"(50, 60]",27
"(60, 70]",15
"(70, 80]",6
"(80, 90]",2


In [59]:
j = ((rcount_age / age_count)*100).set_index(age_count.index).rename(columns={'count':'% of recovered'})
np.round(j, decimals=2)

,% of recovered
age,
"(0, 10]",52.63
"(10, 20]",20.00
"(20, 30]",20.00
"(30, 40]",22.15
"(40, 50]",21.37
"(50, 60]",14.75
"(60, 70]",11.19
"(70, 80]",8.33
"(80, 90]",6.06


<h4> gender </h4>

In [76]:
#dead.groupby(['gender']).count()
rg = pd.DataFrame(r.groupby('gender')['id'].nunique()).rename(columns={'id':'count'})
rg

,count
gender,
female,66
male,87


In [78]:
g2 = ((rg/tg)*100).set_index(tg.index).rename(columns={'count':'% of recovered'})
g2

,% of recovered
gender,
female,17.277487
male,16.730769
